# Process @jongraz tiktok stats for Noodle videos

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Read in our hand collected list of Noodle videos

In [3]:
src = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQz8eEafXnIeKsNvGL8M56Ia6vu89JDCjfJ22ORl65So4kRYtmTbwmqLjE223fe4mjcZqkrs-KT6j8z/pub?\
gid=0&single=true&output=csv"
)

### Create an ID field from the URL

In [4]:
src['id'] = src['url'].str.replace('https://www.tiktok.com/@jongraz/video/', "", regex=False).astype(str)

### Make a list for subsetting the dataframe later

In [5]:
bones_videos = src['id'].to_list()

### Have to run this manually -- for now

In [6]:
# Get cookie and run this...

In [7]:
# !python '/Users/stiles/github/noodle-tracker/_workspace/grab_user_stats.py'

---

### Read scraped stats data

In [8]:
df = pd.read_csv('data/processed/jongraz_videos.csv', dtype={'video_id':str})

### Clean update dates, times

In [9]:
df['timestamp'] = pd.to_datetime(df['video_time'], unit='s')
df['date'] = pd.to_datetime(df['video_time'], unit='s').dt.date
df['month'] = pd.to_datetime(df['video_time'], unit='s').dt.month
df['weekday'] = pd.to_datetime(df['video_time'], unit='s').dt.day_name()
df['time'] = pd.to_datetime(df['video_time'], unit='s').dt.time
df['weekstart_timestamp'] = pd.to_datetime(df['video_time'], unit='s') - pd.to_datetime(df['video_time'], unit='s').dt.weekday * np.timedelta64(1, 'D')
df['weekstart'] = df['weekstart_timestamp'].dt.date
df['updated_date'] = pd.to_datetime("today")

### How many videos? 

In [10]:
len(df)

172

### Select only "bones" videos from @jongraz's feed

In [11]:
bones_df = df[df['video_id'].isin(bones_videos)]

### How many are Noodle videos? 

In [12]:
len(bones_df)

74

In [13]:
bones_df.head()

,user_name,video_id,video_desc,video_time,video_duration,n_shares,n_likes,n_comments,n_plays,video_link,timestamp,date,month,weekday,time,weekstart_timestamp,weekstart,updated_date
0,jongraz,7056051376929066287,message received loud and CLEAR 🔮🦴🔮 #nobones #noodletok #saturdayvibes #pug,1642864985,38,3052,71900,1549,267600,https://www.tiktok.com/@jongraz/video/7056051376929066287?lang=en,2022-01-22 15:23:05,2022-01-22,1,Saturday,15:23:05,2022-01-17 15:23:05,2022-01-17,2022-01-23 08:08:20.778424
1,jongraz,7055678935107030319,buckle up everybody 🔮🦴🔮,1642778270,57,2737,55200,1206,235900,https://www.tiktok.com/@jongraz/video/7055678935107030319?lang=en,2022-01-21 15:17:50,2022-01-21,1,Friday,15:17:50,2022-01-17 15:17:50,2022-01-17,2022-01-23 08:08:20.778424
2,jongraz,7054939278937165102,#greenscreen a bones day FOR SURE! thank you all so much for everything 😭 the book comes out June 7 - preorder link in my bio! #noodlesupremacy,1642606056,58,4452,92900,3314,372200,https://www.tiktok.com/@jongraz/video/7054939278937165102?lang=en,2022-01-19 15:27:36,2022-01-19,1,Wednesday,15:27:36,2022-01-17 15:27:36,2022-01-17,2022-01-23 08:08:20.778424
5,jongraz,7053837902991658287,his message today is…CLEAR 🦴🔮🦴,1642349620,59,3011,70100,1569,325000,https://www.tiktok.com/@jongraz/video/7053837902991658287?lang=en,2022-01-16 16:13:40,2022-01-16,1,Sunday,16:13:40,2022-01-10 16:13:40,2022-01-10,2022-01-23 08:08:20.778424
8,jongraz,7053454719641029934,you heard him! 🔮🦴🔮,1642260404,50,5730,146900,2357,571300,https://www.tiktok.com/@jongraz/video/7053454719641029934?lang=en,2022-01-15 15:26:44,2022-01-15,1,Saturday,15:26:44,2022-01-10 15:26:44,2022-01-10,2022-01-23 08:08:20.778424


---

### Download cover art

In [14]:
# for v,u in zip(bones_df['video_cover'], bones_df['video_id']):
#     !wget {v} -O ../assets/images/noodle_covers/{u}

---

### Weekly plays

In [15]:
weekly_plays = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_plays':sum}).reset_index()
weekly_plays.dtypes

weekstart    object
n_plays       int64
dtype: object

In [16]:
weekly_plays['weekstart'] = pd.to_datetime(weekly_plays['weekstart'])

In [17]:
alt.Chart(weekly_plays).mark_bar(width=20).encode(
    x='weekstart:T',
    y='n_plays'
).properties(width=630)

alt.Chart(...)

---

### Weekly likes

In [18]:
bones_df.head()

,user_name,video_id,video_desc,video_time,video_duration,n_shares,n_likes,n_comments,n_plays,video_link,timestamp,date,month,weekday,time,weekstart_timestamp,weekstart,updated_date
0,jongraz,7056051376929066287,message received loud and CLEAR 🔮🦴🔮 #nobones #noodletok #saturdayvibes #pug,1642864985,38,3052,71900,1549,267600,https://www.tiktok.com/@jongraz/video/7056051376929066287?lang=en,2022-01-22 15:23:05,2022-01-22,1,Saturday,15:23:05,2022-01-17 15:23:05,2022-01-17,2022-01-23 08:08:20.778424
1,jongraz,7055678935107030319,buckle up everybody 🔮🦴🔮,1642778270,57,2737,55200,1206,235900,https://www.tiktok.com/@jongraz/video/7055678935107030319?lang=en,2022-01-21 15:17:50,2022-01-21,1,Friday,15:17:50,2022-01-17 15:17:50,2022-01-17,2022-01-23 08:08:20.778424
2,jongraz,7054939278937165102,#greenscreen a bones day FOR SURE! thank you all so much for everything 😭 the book comes out June 7 - preorder link in my bio! #noodlesupremacy,1642606056,58,4452,92900,3314,372200,https://www.tiktok.com/@jongraz/video/7054939278937165102?lang=en,2022-01-19 15:27:36,2022-01-19,1,Wednesday,15:27:36,2022-01-17 15:27:36,2022-01-17,2022-01-23 08:08:20.778424
5,jongraz,7053837902991658287,his message today is…CLEAR 🦴🔮🦴,1642349620,59,3011,70100,1569,325000,https://www.tiktok.com/@jongraz/video/7053837902991658287?lang=en,2022-01-16 16:13:40,2022-01-16,1,Sunday,16:13:40,2022-01-10 16:13:40,2022-01-10,2022-01-23 08:08:20.778424
8,jongraz,7053454719641029934,you heard him! 🔮🦴🔮,1642260404,50,5730,146900,2357,571300,https://www.tiktok.com/@jongraz/video/7053454719641029934?lang=en,2022-01-15 15:26:44,2022-01-15,1,Saturday,15:26:44,2022-01-10 15:26:44,2022-01-10,2022-01-23 08:08:20.778424


In [19]:
weekly_likes = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_likes':sum}).reset_index()
weekly_likes.dtypes

weekstart    object
n_likes       int64
dtype: object

In [20]:
weekly_likes['weekstart'] = pd.to_datetime(weekly_likes['weekstart'])

In [21]:
alt.Chart(weekly_likes).mark_bar(width=20).encode(
    x='weekstart:T',
    y='n_likes'
).properties(width=630)

alt.Chart(...)

---

### Weekly comments

In [22]:
weekly_comments = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_comments':sum}).reset_index()
weekly_comments.dtypes

weekstart     object
n_comments     int64
dtype: object

In [23]:
weekly_comments['weekstart'] = pd.to_datetime(weekly_comments['weekstart'])

In [24]:
alt.Chart(weekly_comments).mark_bar(width=20).encode(
    x='weekstart:T',
    y='n_comments'
).properties(width=630)

alt.Chart(...)

---

### Exports

In [25]:
weekly_plays.to_csv("../_data/weekly_plays.csv", index=False)
weekly_plays.to_csv("../assets/data/weekly_plays.csv", index=False)

In [26]:
weekly_likes.to_csv("../_data/weekly_likes.csv", index=False)
weekly_likes.to_csv("../assets/data/weekly_likes.csv", index=False)

In [27]:
weekly_comments.to_csv("../_data/weekly_comments.csv", index=False)
weekly_comments.to_csv("../assets/data/weekly_comments.csv", index=False)